In [329]:
import xarray as xr
import xesmf as xe
from numpy import loadtxt
import pandas as pd
import glob
import numpy as np
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score, recall_score, f1_score
from tqdm import tqdm

In [330]:
#Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [331]:
#Return bounding box of prometeus input
def get_bbox(df):
    list = []
    
    list.append(df["XLAT"].min().values)
    list.append(df["XLAT"].max().values)
    list.append(df["XLONG"].min().values)
    list.append(df["XLONG"].max().values)
    
    return list

#Return remapped dataset to specific spatial resolution
def remap(df, bbox):
    global gridsize
        
    ds_out = xe.util.grid_2d(bbox[2], bbox[3], gridsize, bbox[0], bbox[1], gridsize)        
    
    #Define a new grid with bbox and gridsize 
    #Determining time variable name
    if "time" in list(df.dims):
        regridder = xe.Regridder(df.isel(time=0), ds_out, "bilinear")
    else:
        regridder = xe.Regridder(df.isel(Time=0), ds_out, "bilinear")
    
    #Remapping input data with new grid specifications
    out = regridder(df)
    
    return out

#Return nearest point from dataset in x,y format 
def get_nearest_point(ds, lat, lon): 
    abslat = np.abs(ds.lat-lat)
    abslon = np.abs(ds.lon-lon)
    
    c = np.maximum(abslon, abslat)    
    
    ([xloc], [yloc]) = np.where(c==np.min(c))
        
    return xloc, yloc

In [337]:
#Read nc files, extract, transform and return resampled dataset
#Works with any wrfout files
def get_wrf_df(date, source, drop, bpath, rpath, bname):    
    global lat, lon, bbox
    
    wrf_date = pd.to_datetime(date).strftime("%Y-%m-%d_%H:%M:%S")
    
    #Define full path of file
    full_path = bpath + rpath + date + bname + wrf_date

    #Open dataset with xarray
    df = xr.open_dataset(full_path)

    #Drop unused variables
    df = df.drop(drop)
    
    #Get bbox of prometeus lat_min, lat_max, lon_min, lon_max
#     bbox = get_bbox(df)    
    
    #Modify Times format, from byte to datetime
    time_strs = [str(i.values)[1:].replace("_"," ") for i in df["Times"]]
    time_datetime = pd.to_datetime(time_strs)
    
    #Modify Time dimension to time (standar name in lowercase)
    ds_wrf_timedim = df.rename({'Time':'time'})
    
    #Asign new time format strings 
    ds_wrf_timecoord = ds_wrf_timedim.assign(time=time_datetime)
    
    #Drop Times varible
    df = ds_wrf_timecoord.drop('Times')

    #Select only 24 hours from input
    df = df.sel(time=slice(date))
    
    #Remap to new gridisze    
    remapped_df = remap(df, bbox)    
    
    #Eliminar los index y asignar time como nuevo, y convertirlo a datetime
    df_out = remapped_df.to_dataframe().reset_index()
    
    #Rename column time to date 
    df_out = df_out.rename({'time':'date'}, axis='columns')
    
    df_out["RAIN"] = df_out["RAINC"] + df_out["RAINNC"]
    df_out[source] = df_out.groupby(['x', 'y'])['RAIN'].diff().fillna(df_out["RAIN"][0])
    df_out[source] = df_out[source].round(3)
    df_out.drop(columns=["RAIN", "RAINC", "RAINNC"], inplace=True)
    
    return df_out

#Read GPM files, extract, transform and return resampled dataset
#Works with any Global Precipitation Measurement dataset
def get_gpm_df(date, drop, bpath, rpath):
    global lat, lon, bbox
    
    wrf_date = pd.to_datetime(date).strftime("%Y-%m-%d_%H:%M:%S") 

    #Define full path of GPM files
    full_path = bpath + rpath

    #Merge al GPM files of current date
    df = xr.merge([xr.open_dataset(f) for f in glob.glob(f'{full_path}/*{date}*')])
    
    #Drop unused variables
    df = df.drop(drop)
    
    #Remap to new gridisze    
    remapped_df = remap(df, bbox)  
    
    #Eliminar los index y asignar time como nuevo, y convertirlo a datetime
    df_out = remapped_df.to_dataframe().reset_index()
    
    df_out['time'] = pd.to_datetime(df_out['time'])
    df_out.set_index("time", inplace=True)
    
    #Resample data
    df_resampled = df_out.groupby(["x", "y", pd.Grouper(freq='h')]).agg({'precipitationCal': 'sum', 'lat':'last', 'lon':'last'})        
    df_resampled
    
    #Rename columns
    df_resampled.rename(columns={"precipitationCal": "gpm"}, inplace=True)        
    df_resampled.rename(columns={"time": "date"}, inplace=True)
    df_resampled["gpm"] = df_resampled["gpm"].round(3)            

    df_resampled = df_resampled.reset_index()
    df_resampled = df_resampled.rename({'time':'date'}, axis='columns')

    return df_resampled


In [338]:
#Bounding Box of prometeus wrf output at 1.8km resolution
#For a different wrfout use get_bbox function defined before
bbox = [26.93850708, 30.97611237, -112.75549316, -109.09912109]

#Gridsize in degrees, aprox 3Km
gridsize = 0.027

#Base path
bpath = "/LUSTRE/home/daniel/"

#Conagua wrfout path
con_rpath = "namelist_conagua/salidas/"
#Conagua output file name
con_bname = "/wrfout_d01_"

#Prometeus wrfout path
pro_rpath = "namelist_prometeus/salidas/"
#Prometeus output file name
pro_bname = "/wrfout_d02_"

#GPM files path
gpm_rpath = "namelist_prometeus/gpm_imerg/"

#Drop unused variables
drop_conagua = ["T", "T2"]
drop_prometeus = ["T2"]
drop_gpm = ["precipitationUncal", "randomError", "HQprecipitation", "HQprecipSource", 
            "IRprecipitation", "HQobservationTime", "IRkalmanFilterWeight", 
            "probabilityLiquidPrecipitation", "precipitationQualityIndex"]


In [340]:
#fechas is a file that contains dates of storm events on northwest of México
dates_file = loadtxt("fechas", dtype="str")

#Define empty dataframes
gpm = pd.DataFrame()
conagua = pd.DataFrame()
prometeus = pd.DataFrame()

#Get datasets from conagua, prometus and gpm, then append to specific dataframe
for date in tqdm(dates_file):
    ngpm = get_gpm_df(str(date), drop_gpm, bpath, gpm_rpath)
    gpm = gpm.append(ngpm, ignore_index=True)

    nconagua = get_wrf_df(str(date), "conagua", drop_conagua, bpath, con_rpath, con_bname)
    conagua = conagua.append(nconagua, ignore_index=True)
    
    nprometeus = get_wrf_df(str(date), "prometeus", drop_prometeus, bpath, pro_rpath, pro_bname)
    prometeus = prometeus.append(nprometeus, ignore_index=True)
    
#Merge all dataframes into result dataframe
# result = pd.merge(conagua, prometeus, how='left', on=['date', 'lat', 'lon'])
# result = pd.merge(result, gpm, how='left', on=['date', 'lat', 'lon'])

#Save it to merged_info.csv file
# result.to_csv("merged_info.csv", index=False)
gpm.to_csv("gpm_all_points.csv", index=False)
prometeus.to_csv("prometeus_all_points.csv", index=False)
conagua.to_csv("conagua_all_points.csv", index=False)


100%|██████████| 27/27 [09:19<00:00, 20.71s/it]


In [347]:
conagua["conagua"].describe().apply("{0:.3f}".format)

count    13219200.000
mean            0.551
std             1.773
min             0.000
25%             0.000
50%             0.000
75%             0.282
max            89.777
Name: conagua, dtype: object

In [350]:
prometeus["prometeus"].describe().apply("{0:.3f}".format)

count    13219200.000
mean            0.579
std             3.109
min             0.000
25%             0.000
50%             0.000
75%             0.000
max           158.706
Name: prometeus, dtype: object

In [351]:
gpm["gpm"].describe().apply("{0:.3f}".format)

count    13219200.000
mean            0.654
std             2.442
min             0.000
25%             0.000
50%             0.000
75%             0.128
max           146.194
Name: gpm, dtype: object